In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import re

import dill
import pandas as pd
import sys
from pathlib import Path

# Adjust this to point to the directory containing `unimib_snowit_project`
project_root = Path.cwd().parent
sys.path.append(str(project_root))

import src.unimib_snowit_project.utils as u

# Setup

In [14]:
# Base Params

DATA_IN_DIR = 'data_input'

USERS_IN_FILENAME = 'users.csv'
PROFILES_IN_FILENAME = 'profiles.csv'
CARDS_IN_FILENAME = 'cards.csv'
ORDERS_IN_FILENAME = 'orders.csv'
ORDER_DETAILS_IN_FILENAME = 'order_details.csv'

DATA_PKL_DIR = 'data_loaded'

USERS_PKL_FILENAME = 'users.pkl'
PROFILES_PKL_FILENAME = 'profiles.pkl'
CARDS_PKL_FILENAME = 'cards.pkl'
ORDERS_PKL_FILENAME = 'orders.pkl'
ORDER_DETAILS_PKL_FILENAME = 'order_details.pkl'

NA_VALUES = ['', ' ', '""',
             '#N/A', '#N/A N/A', '#NA', 'N/A', '<NA>', 'n/a', # 'NA',
             '-1.#IND', '1.#IND',
             '-1.#QNAN', '-NaN', '-nan', '-NAN', '1.#QNAN', 'NaN', 'nan', 'NAN',
             'NULL', 'Null', 'null',
             'NONE', 'None', 'none',
             ]

In [15]:
# Base paths

root_dir_path = u.get_root_dir()

data_in_dir_path = root_dir_path.joinpath(DATA_IN_DIR)
users_in_path = data_in_dir_path.joinpath(USERS_IN_FILENAME)
profiles_in_path = data_in_dir_path.joinpath(PROFILES_IN_FILENAME)
cards_in_path = data_in_dir_path.joinpath(CARDS_IN_FILENAME)
orders_in_path = data_in_dir_path.joinpath(ORDERS_IN_FILENAME)
order_details_in_path = data_in_dir_path.joinpath(ORDER_DETAILS_IN_FILENAME)

data_pkl_dir_path = root_dir_path.joinpath(DATA_PKL_DIR)
users_pkl_path = data_pkl_dir_path.joinpath(USERS_PKL_FILENAME)
profiles_pkl_path = data_pkl_dir_path.joinpath(PROFILES_PKL_FILENAME)
cards_pkl_path = data_pkl_dir_path.joinpath(CARDS_PKL_FILENAME)
orders_pkl_path = data_pkl_dir_path.joinpath(ORDERS_PKL_FILENAME)
order_details_pkl_path = data_pkl_dir_path.joinpath(ORDER_DETAILS_PKL_FILENAME)

# LOAD

## Load Users

In [16]:
safeload_users_df = pd.read_csv(users_in_path,
                                dtype='string',
                                na_values=[],
                                keep_default_na=False
                                )

In [17]:
safeload_users_df.columns

Index(['user.uid', 'createdAt', 'source', 'isAnonymous', 'referralsCount',
       'city', 'language', 'googleId', 'appleId', 'facebookId',
       'referral.medium', 'referral.source', 'referral.type',
       'favouriteZones'],
      dtype='object')

In [18]:
# col_to_check = 'favouriteZones'
# safeload_users_df[col_to_check].drop_duplicates()

In [19]:
# Read and fix
users_df = pd.read_csv(users_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES,
                       dtype={
                           'user.uid': 'string',
                           'createdAt': 'string',
                           'source': 'string',
                           'isAnonymous': 'boolean',
                           'referralsCount': 'Int64',
                           'city': 'string',
                           'language': 'string',
                           'googleId': 'boolean',
                           'appleId': 'boolean',
                           'facebookId': 'boolean',
                           'referral.medium': 'string',
                           'referral.source': 'string',
                           'referral.type': 'Int64',
                           'favouriteZones': 'string'
                       }
                       )

users_df['createdAt'] = pd.to_datetime(users_df['createdAt'])

users_df['city'] = (users_df['city']
                    .apply(lambda x:
                           u.clean_str(x, 'lower')
                           if pd.notnull(x)
                           else None
                           )
                    )

users_df['referral.medium'] = (users_df['referral.medium']
                        .apply(lambda x:
                               u.clean_str(x, 'lower')
                               if pd.notnull(x)
                               else None
                               )
                        )

users_df['referral.source'] = (users_df['referral.source']
                        .apply(lambda x:
                               u.clean_str(x, 'lower')
                               if pd.notnull(x)
                               else None
                               )
                        )

users_df['favouriteZones'] = (users_df['favouriteZones']
                              .apply(lambda x:
                                     u.get_list_from_str(x)
                                     if pd.notnull(x)
                                     else []
                                     )
                              )

In [20]:
# CHECK PK VALIDITY

# SELECT count(1) as num_rows
# FROM users_df
# WHERE user.uid IS NULL

display(
    users_df
    .loc[lambda tbl: tbl['user.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

# SELECT user.uid, count(1) as num_rows
# FROM users_df
# GROUP BY user.id
# HAVING num_rows > 1

display(
    users_df
    .assign(aux=1.0)
    .groupby(['user.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

0

,num_rows
user.uid,


## Load Profiles

In [21]:
safeload_profiles_df = pd.read_csv(profiles_in_path,
                                   dtype='string',
                                   na_values=[],
                                   keep_default_na=False
                                   )

In [22]:
safeload_profiles_df.columns

Index(['user.uid', 'profile.uid', 'birthday', 'sex', 'city', 'height',
       'weight', 'skibootsSize', 'level', 'types'],
      dtype='object')

In [23]:
# col_to_check = 'types'
# safeload_profiles_df[col_to_check].drop_duplicates()

In [24]:
# Read and fix
profiles_df = pd.read_csv(profiles_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES,
                       dtype={
                           'user.uid': 'string',
                           'profile.uid': 'string',
                           'birthday': 'string',
                           'sex': 'string',
                           'city': 'string',
                           'height': 'Float64',
                           'weight': 'Float64',
                           'skibootsSize': 'Float64',
                           'level': 'string',
                           'types': 'string'
                       }
                       )

profiles_df['birthday'] = pd.to_datetime(profiles_df['birthday'])

def clean_profile_sex(sex: str) -> str | None:
    clean = u.clean_str(sex, 'upper')
    if clean in ['M', 'F']:
        return clean
    elif clean == ['UOMO', 'MASCHIO']:
        return 'M'
    elif clean == ['DONNA', 'FEMMINA']:
        return 'F'
    else:
        None
profiles_df['sex'] = (profiles_df['sex']
                      .apply(lambda x:
                             clean_profile_sex(x)
                             if pd.notnull(x)
                             else None
                             )
                      )

profiles_df['city'] = (profiles_df['city']
                    .apply(lambda x:
                           u.clean_str(x, 'lower')
                           if pd.notnull(x)
                           else None
                           )
                    )
profiles_df['types'] = (profiles_df['types']
                              .apply(lambda x:
                                     u.get_list_from_str(x)
                                     if pd.notnull(x)
                                     else []
                                     )
                              )

In [25]:
# CHECK PK VALIDITY

# SELECT count(1) as num_rows
# FROM profiles_df
# WHERE profile.uid IS NULL

display(
    profiles_df
    .loc[lambda tbl: tbl['profile.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

# SELECT profile.uid, count(1) as num_rows
# FROM profiles_df
# GROUP BY profile.id
# HAVING num_rows > 1

display(
    profiles_df
    .assign(aux=1.0)
    .groupby(['profile.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

0

,num_rows
profile.uid,


In [26]:
# CHECK FK VALIDITY

# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM profiles_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# GROUP BY in_users
# HAVING num_rows > 1

(profiles_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_users'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,46026
NaN,18212


In [27]:
(profiles_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .loc[lambda tbl: tbl['in_users'].isnull()]
)

,user.uid,in_users
0,tvvgjd4sdgfucn33brkigrnjjv,NaN
2,bmgilq4eatrpt2hhwamnbcqnir,NaN
3,ysbrr5ve9azqvr1jw72expbnxr,NaN
4,txy3ls535wzclygjnlt2gjalm1,NaN
5,f73bpmavwla9pho2qpgi5n7cdx,NaN
...,...,...
64149,mtlnpaw38weptp9pmbbyifknko,NaN
64164,jpkst6ew69gkluh7in43m4ohvs,NaN
64166,smqgdbfhyofngnjqth4bglwax7,NaN
64217,pzffry3ps6aagidfvisp89giam,NaN


In [28]:
profile_fail_useruids = (profiles_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .loc[lambda tbl: tbl['in_users'].isnull()]
    ['user.uid']
)

display(profile_fail_useruids)

profile_fail_useruid_df = profiles_df.loc[lambda tbl: tbl['user.uid'].isin(profile_fail_useruids)]

display(profile_fail_useruid_df)

0        tvvgjd4sdgfucn33brkigrnjjv
2        bmgilq4eatrpt2hhwamnbcqnir
3        ysbrr5ve9azqvr1jw72expbnxr
4        txy3ls535wzclygjnlt2gjalm1
5        f73bpmavwla9pho2qpgi5n7cdx
                    ...            
64149    mtlnpaw38weptp9pmbbyifknko
64164    jpkst6ew69gkluh7in43m4ohvs
64166    smqgdbfhyofngnjqth4bglwax7
64217    pzffry3ps6aagidfvisp89giam
64219    34w1omc2f7fskwlhteltqcomrc
Name: user.uid, Length: 18212, dtype: string

,user.uid,profile.uid,birthday,sex,city,height,weight,skibootsSize,level,types
0,tvvgjd4sdgfucn33brkigrnjjv,9oqhfw0zsc5prlkwh9gk,2000-05-23,M,zwolle,180.0,70.0,43.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
2,bmgilq4eatrpt2hhwamnbcqnir,vi88jcuymkmxkandvy3l,1992-05-06,F,z√ºrich,<NA>,<NA>,<NA>,intermediate,"[""insurance"", ""rent/bike"", ""rent"", ""lesson"", ""..."
3,ysbrr5ve9azqvr1jw72expbnxr,p54s6m0hejd4nucibz4e,1998-02-16,F,zurich,161.0,56.0,38.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
4,ysbrr5ve9azqvr1jw72expbnxr,zkzpf8fnha6s4kmc2rwn,1994-05-12,M,zurich,180.0,80.0,45.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
5,txy3ls535wzclygjnlt2gjalm1,bqnrhm4itmwhofvxxxqh,1979-12-09,M,zur moshe,180.0,75.0,43.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
...,...,...,...,...,...,...,...,...,...,...
97043,smqgdbfhyofngnjqth4bglwax7,ib1un3kn23p8dn5jso3e,2018-09-04,M,albosaggia,118.0,20.0,29.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
97044,smqgdbfhyofngnjqth4bglwax7,ah904epxoefqvk2slt6k,2018-09-04,M,albosaggia,118.0,20.0,29.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
97097,dyz1ivjmcym14jagwuyjnhtrz7,mdqipvzojgtgsajxjtlx,1999-10-05,None,aidone,155.0,55.0,36.0,<NA>,"[""insurance"", ""rent/bike"", ""rent"", ""travel"", ""..."
97126,pzffry3ps6aagidfvisp89giam,mbpgeblzb2t0bylyrhww,2003-10-17,M,acquedolci,167.0,60.0,39.0,intermediate,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."


In [29]:
# FIX FK ISSUE
profiles_df.drop(list(profile_fail_useruid_df.index), inplace=True)

In [30]:
display(
    profiles_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .assign(aux=1.0)
    .groupby(['in_users'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,46026


## Load Cards

In [31]:
safeload_cards_df = pd.read_csv(cards_in_path,
                                dtype='string',
                                na_values=[],
                                keep_default_na=False
                                )

In [32]:
safeload_cards_df.columns

Index(['card.uid', 'assignedAt', 'birthday', 'status', 'user.uid'], dtype='object')

In [33]:
# ...

In [34]:
# Load Cards with typed schema
cards_df = pd.read_csv(cards_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES,
                       dtype={
                           'card.uid': 'string',
                           'user.uid': 'string',
                           'assignedAt': 'string',
                           'birthday': 'string',
                           'status': 'string'
                       })

# Convert to datetime
cards_df['assignedAt'] = pd.to_datetime(cards_df['assignedAt'], errors='coerce')
cards_df['birthday'] = pd.to_datetime(cards_df['birthday'], errors='coerce')

# Normalize strings
cards_df['status'] = cards_df['status'].apply(lambda x: u.clean_str(x, 'lower') if pd.notnull(x) else None)
cards_df['user.uid'] = cards_df['user.uid'].apply(lambda x: u.clean_str(x, 'lower') if pd.notnull(x) else None)



In [35]:
# CHECK PK VALIDITY
print("🔍 Null card.uid count:", cards_df['card.uid'].isnull().sum())
display(
    cards_df
    .assign(aux=1)
    .groupby('card.uid')
    .agg(num_rows=('aux', 'count'))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

🔍 Null card.uid count: 0


,num_rows
card.uid,
05665255,2
05665266,2
05665277,2
05665288,2
05665299,2
...,...
06859428,2
06859439,2
06859440,2


In [36]:
cards_df.drop_duplicates(inplace=True)
# Check PK validity after deleting
print("🔍 Null card.uid count:", cards_df['card.uid'].isnull().sum())
display(
    cards_df
    .assign(aux=1)
    .groupby('card.uid')
    .agg(num_rows=('aux', 'count'))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

🔍 Null card.uid count: 0


,num_rows
card.uid,


In [37]:
cards_df['is_fully_empty_not_assigned'] = (
    (cards_df['status'] == 'not-assigned') &
    (cards_df['user.uid'].replace('', pd.NA).isna()) &
    (cards_df['assignedAt'].isna()) &
    (cards_df['birthday'].isna())
)

#Group and display as a summary table
display(
    cards_df
    .loc[cards_df['is_fully_empty_not_assigned']]
    .assign(aux=1)
    .groupby('status')
    .agg(num_rows=('aux', 'sum'))
)


,num_rows
status,
not-assigned,440808


In [38]:
cards_df = cards_df.loc[~cards_df['is_fully_empty_not_assigned']].copy()
display(
    cards_df
    .loc[cards_df['is_fully_empty_not_assigned']]
    .assign(aux=1)
    .groupby('status')
    .agg(num_rows=('aux', 'sum'))
)

,num_rows
status,


In [39]:
# Find invalid user.uids in cards_df not present in users_df
invalid_user_uids = cards_df[~cards_df['user.uid'].isin(users_df['user.uid'])]

# Show the invalid records
display(invalid_user_uids)


,card.uid,assignedAt,birthday,status,user.uid,is_fully_empty_not_assigned
0,00000001,2025-02-01,2018-07-02,pending,6m8fx3mvanac7bcltkx79lbxjs,False
7,00000077,2017-12-20,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6,False
8,00000088,2018-01-07,NaT,pending,mlwkfozqeefazgmboia8hxxkj1,False
12,00000125,2020-01-02,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6,False
13,00000136,2020-01-19,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6,False
...,...,...,...,...,...,...
805347,99127132,2022-12-29,1996-02-21,pending,nvezbqfonuzshthd38g3ajjopo,False
805369,99127350,2024-12-27,2016-01-01,valid,oa88nzwvnfqifqhhuegnizcuzk,False
805736,99131027,2024-01-19,1965-04-23,pending,abpz5y53frcqdfc9nl5ojj1uzi,False
805806,99131728,2022-01-28,1995-10-14,pending,xtaom20duhpf4r3mrkxks12ifk,False


In [40]:
# Drop rows with invalid user.uid
cards_df = cards_df.loc[cards_df['user.uid'].isin(users_df['user.uid'])].copy()

# Check again - should be empty now
invalid_user_uids = cards_df[~cards_df['user.uid'].isin(users_df['user.uid'])]

display(invalid_user_uids)

,card.uid,assignedAt,birthday,status,user.uid,is_fully_empty_not_assigned


In [41]:
valid_statuses = [
    'error',
    'not-assigned',
    'pending',
    'warning',
    'valid',
    'rejected',
    'membership'
]

# Check rows with invalid status values
invalid_status_rows = cards_df.loc[~cards_df['status'].isin(valid_statuses)]

print(f"Number of invalid status rows: {len(invalid_status_rows)}")

# Display invalid status rows if any
display(invalid_status_rows)

Number of invalid status rows: 0


,card.uid,assignedAt,birthday,status,user.uid,is_fully_empty_not_assigned


## Load Orders

In [42]:
safeload_orders_df = pd.read_csv(orders_in_path,
                                 dtype='string',
                                 na_values=[],
                                 keep_default_na=False
                                )

In [43]:
safeload_orders_df.columns

Index(['order.uid', 'user.uid', 'createdAt', 'createdAtTime', 'paymentGateway',
       'paymentBrand', 'pickup', 'pickupComplete', 'source', 'tenant',
       'paymentAttempts', 'timeZone', 'clientInfo'],
      dtype='object')

In [44]:
# col_to_check = 'clientInfo'
# safeload_orders_df[col_to_check].drop_duplicates()

In [45]:
# Read and fix
orders_df = pd.read_csv(orders_in_path,
                        dtype='string',
                        keep_default_na=False,
                        na_values=NA_VALUES)

# Datetime fields
orders_df['createdAt'] = pd.to_datetime(orders_df['createdAt'], errors='coerce').dt.normalize()
orders_df['createdAtTime'] = pd.to_datetime(orders_df['createdAtTime'], errors='coerce')

# Booleans
orders_df['pickup'] = orders_df['pickup'].map({'True': True, 'False': False})
orders_df['pickupComplete'] = orders_df['pickupComplete'].map({'True': True, 'False': False})

# Normalize strings
str_cols = ['paymentGateway', 'paymentBrand', 'source', 'tenant', 'timeZone']
for col in str_cols:
    orders_df[col] = orders_df[col].apply(lambda x: u.clean_str(x, 'lower') if pd.notnull(x) else None)

orders_df['user.uid'] = orders_df['user.uid'].apply(lambda x: u.clean_str(x, 'lower') if pd.notnull(x) else None)

# Numeric
orders_df['paymentAttempts'] = pd.to_numeric(orders_df['paymentAttempts'], errors='coerce')

import json # Used to decode dict type
# JSON decode clientInfo
def parse_client_info(x):
    try:
        return json.loads(x)
    except Exception:
        return None

orders_df['clientInfo'] = orders_df['clientInfo'].apply(parse_client_info)


In [46]:
# PRIMARY KEY CHECK
print("🔍 Null order.uid count:", orders_df['order.uid'].isnull().sum())
display(
    orders_df
    .assign(aux=1)
    .groupby('order.uid')
    .agg(num_rows=('aux', 'count'))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

🔍 Null order.uid count: 0


,num_rows
order.uid,


In [47]:
# Find invalid user.uids in orders_df not present in users_df
invalid_user_uids = orders_df[~orders_df['user.uid'].isin(users_df['user.uid'])]

# Show the invalid records
display(invalid_user_uids)

,order.uid,user.uid,createdAt,createdAtTime,paymentGateway,paymentBrand,pickup,pickupComplete,source,tenant,paymentAttempts,timeZone,clientInfo
1,termljwsr2gecnzwks,ghol7lwjlhewwvasejlxa9fv00,2024-07-01,2024-07-01 18:34:12,discount,None,False,False,website,snowit_esterno,1,None,"{'browser': {'major': '126', 'name': 'Chrome',..."
2,ko8axscf5grobnz9in,nhuh3scqoktj6n5mnqkvgyh43h,2024-07-01,2024-07-01 16:08:26,discount,None,False,False,website,snowit_esterno,1,None,"{'browser': {'major': '126', 'name': 'Chrome',..."
3,tu8b9haokkimu72aza,oyekz0wyxcplvrpencpuue6sdt,2024-07-01,2024-07-01 16:02:11,stripe,mastercard,False,False,website,snowit_esterno,2,None,"{'browser': {'major': '126', 'name': 'Chrome',..."
4,a8mvwuv6ileh9aqblu,1p0ocryxz4ts2wtsi1bawtjndt,2024-07-01,2024-07-01 15:46:12,stripe,None,False,False,website,tribala,2,None,"{'browser': {'major': '125', 'name': 'Chrome',..."
5,kmwabdpnhtm46p8me8,nhuh3scqoktj6n5mnqkvgyh43h,2024-07-01,2024-07-01 09:01:16,discount,None,False,False,website,snowit_esterno,1,None,"{'browser': {'major': '126', 'name': 'Chrome',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549883,bjzuut5lvj76ad6jbp,wnnh4uxef0m2il1kys1xbogkry,2022-07-01,2022-07-01 21:02:41,discount,None,False,False,website,snowit_esterno,1,europe/rome,"{'browser': {'major': '102', 'name': 'Chrome',..."
549884,ur0t4p0hejetoesg3h,wnnh4uxef0m2il1kys1xbogkry,2022-07-01,2022-07-01 20:56:39,discount,None,False,False,website,snowit_esterno,1,europe/rome,"{'browser': {'major': '102', 'name': 'Chrome',..."
549886,3unzmjqzzmwgnbil56,iz4qy0qs0qre4g8ndczecyms3s,2022-07-01,2022-07-01 19:51:24,stripe,None,False,False,website,snowit_esterno,1,europe/rome,"{'browser': {'major': '103', 'name': 'Chrome',..."
549888,mhosjzeznativvqfnz,ri3lqgshwnyc7tqb1ray1mvllb,2022-07-01,2022-07-01 19:21:13,satispay,None,False,False,website,snowit_esterno,1,europe/rome,"{'browser': {'major': '15', 'name': 'Mobile Sa..."


In [48]:
# Drop rows with invalid user.uid
orders_df = orders_df.loc[orders_df['user.uid'].isin(users_df['user.uid'])].copy()

# Check again - should be empty now
invalid_user_uids = orders_df[~orders_df['user.uid'].isin(users_df['user.uid'])]

display(invalid_user_uids)

,order.uid,user.uid,createdAt,createdAtTime,paymentGateway,paymentBrand,pickup,pickupComplete,source,tenant,paymentAttempts,timeZone,clientInfo


## Load Order Details

In [49]:
safeload_order_details_df = pd.read_csv(order_details_in_path,
                                        dtype='string',
                                        na_values=[],
                                        keep_default_na=False
                                        )

In [50]:
safeload_order_details_df.columns

Index(['item.uid', 'order.uid', 'item.date', 'product.uid',
       'product.dynamicPricing', 'item.amount', 'item.discount',
       'product.type', 'item.zoneName', 'product.durationHours',
       'item.profiles', 'item.variantName', 'item.slotName',
       'item.snowitcardNumber', 'item.status'],
      dtype='object')

In [51]:
# Load with correct dtypes
order_details_df = pd.read_csv(order_details_in_path,
                               keep_default_na=False,
                               na_values=NA_VALUES,
                               dtype={
                                   'item.uid': 'string',
                                   'order.uid': 'string',
                                   'item.status': 'string',
                                   'item.date': 'string',
                                   'product.uid': 'string',
                                   'product.dynamicPricing': 'boolean',
                                   'item.amount': 'float',
                                   'item.discount': 'boolean',
                                   'product.type': 'string',
                                   'item.zoneName': 'string',
                                   'product.durationHours': 'float',
                                   'item.profiles': 'string',
                                   'item.variantName': 'string',
                                   'item.slotName': 'string',
                                   'item.snowitcardNumber': 'string'
                               })

# Date parsing
order_details_df['item.date'] = pd.to_datetime(order_details_df['item.date'], errors='coerce')

# Clean string fields
cols_to_clean = ['item.uid', 'order.uid', 'product.uid', 'product.type', 'item.zoneName',
                 'item.profiles', 'item.variantName', 'item.slotName', 'item.snowitcardNumber', 'item.status']

for col in cols_to_clean:
    order_details_df[col] = order_details_df[col].apply(lambda x: u.clean_str(x, 'lower') if pd.notnull(x) else None)

# Clean negative/invalid prices if needed — optional validation step
# Example: Negative transfers vs valid refunds
# order_details_df = order_details_df[order_details_df['item.amount'] >= 0]


In [52]:
# PRIMARY KEY CHECK
# Check nulls
print("🔍 Null composite key count:", order_details_df[['item.uid', 'order.uid', 'item.status']].isnull().any(axis=1).sum())

# Check duplicates
display(
    order_details_df
    .assign(aux=1)
    .groupby(['item.uid', 'order.uid', 'item.status'], dropna=False)
    .agg(num_rows=('aux', 'count'))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)



🔍 Null composite key count: 0


,,,num_rows
item.uid,order.uid,item.status,


In [53]:
duplicates_df = order_details_df.duplicated(subset=['item.uid', 'order.uid', 'item.status'], keep=False)

# Show duplicates
display(order_details_df[duplicates_df])

,item.uid,order.uid,item.date,product.uid,product.dynamicPricing,item.amount,item.discount,product.type,item.zoneName,product.durationHours,item.profiles,item.variantName,item.slotName,item.snowitcardNumber,item.status


In [54]:
valid_statuses = [
    'fulfilled',
    'ok',
    'on-hold',
    'canceled',
    'transfer',
    'processing']

# Check rows with invalid status values
invalid_status_rows = order_details_df.loc[~order_details_df['item.status'].isin(valid_statuses)]

print(f"Number of invalid status rows: {len(invalid_status_rows)}")

# Display invalid status rows if any
display(invalid_status_rows)

Number of invalid status rows: 0


,item.uid,order.uid,item.date,product.uid,product.dynamicPricing,item.amount,item.discount,product.type,item.zoneName,product.durationHours,item.profiles,item.variantName,item.slotName,item.snowitcardNumber,item.status


In [55]:
# Find negative item.amount
negative_prices_df = order_details_df[order_details_df['item.amount'] < 0]

# Show potentially problematic entries (may represent transfers/refunds)
display(negative_prices_df)

,item.uid,order.uid,item.date,product.uid,product.dynamicPricing,item.amount,item.discount,product.type,item.zoneName,product.durationHours,item.profiles,item.variantName,item.slotName,item.snowitcardNumber,item.status
4,axrcvxzh7evw2ebakhho,ko8axscf5grobnz9in,2024-07-02,9onsf7m6j7bnwakw2otu,False,-26.0,False,food,bardonecchia,4.0,None,None,None,None,transfer
6,bshibhxedpkdtmzpmczi,ko8axscf5grobnz9in,2024-07-02,9onsf7m6j7bnwakw2otu,False,-26.0,False,food,bardonecchia,4.0,None,None,None,None,transfer
7,wf4gfrruh3zdyp15nnql,ko8axscf5grobnz9in,2024-07-02,9onsf7m6j7bnwakw2otu,False,-26.0,False,food,bardonecchia,4.0,None,None,None,None,transfer
9,4neyeg0izzwue8rbjrph,ko8axscf5grobnz9in,2024-07-02,9onsf7m6j7bnwakw2otu,False,-26.0,False,food,bardonecchia,4.0,None,None,None,None,transfer
14,qihrelx8hg0hk8ekxxl6,kmwabdpnhtm46p8me8,2024-07-01,9onsf7m6j7bnwakw2otu,False,-1.0,False,food,bardonecchia,4.0,None,None,None,None,transfer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420575,w02tubwnk9jzv6n3ilfc,9bdixdkb6c5mozsgjd,2022-07-15,xqjvqvqboppg2ujfffpw,False,-11.5,False,skipass,lecco - piani d'erna,12.0,None,None,None,None,transfer
1420584,pkdquhd2s8ydo6khsjrd,h0vb0rmb2mk4c7s3tv,2022-07-15,xqjvqvqboppg2ujfffpw,False,-11.5,False,skipass,lecco - piani d'erna,12.0,None,None,None,None,transfer
1420590,6e6q400crcowunygsgcu,6icnjfrltzhdi2eato,2022-07-15,xqjvqvqboppg2ujfffpw,False,-11.5,False,skipass,lecco - piani d'erna,12.0,None,None,None,None,transfer
1420592,oee4z1zf7sk4vm0prmrw,6icnjfrltzhdi2eato,2022-07-15,xqjvqvqboppg2ujfffpw,False,-11.5,False,skipass,lecco - piani d'erna,12.0,None,None,None,None,transfer


# Save

In [56]:
# Save Cleaned Dataset

with users_pkl_path.open('wb') as fh:
    dill.dump(users_df, fh)
print(f"Save users data in {users_pkl_path.as_posix()}")

with profiles_pkl_path.open('wb') as fh:
    dill.dump(profiles_df, fh)
print(f"Save profiles data in {profiles_pkl_path.as_posix()}")

with cards_pkl_path.open('wb') as fh:
    dill.dump(cards_df, fh)
print(f"Save cards data in {cards_pkl_path.as_posix()}")

with orders_pkl_path.open('wb') as fh:
    dill.dump(orders_df, fh)
print(f"Save orders data in {orders_pkl_path.as_posix()}")

with order_details_pkl_path.open('wb') as fh:
    dill.dump(order_details_df, fh)
print(f"Save order details data in {order_details_pkl_path.as_posix()}")

Save users data in /Users/Simon/Desktop/Marketing_Analytics_project/unimib_snowit_project/data_loaded/users.pkl
Save profiles data in /Users/Simon/Desktop/Marketing_Analytics_project/unimib_snowit_project/data_loaded/profiles.pkl
Save cards data in /Users/Simon/Desktop/Marketing_Analytics_project/unimib_snowit_project/data_loaded/cards.pkl
Save orders data in /Users/Simon/Desktop/Marketing_Analytics_project/unimib_snowit_project/data_loaded/orders.pkl
Save order details data in /Users/Simon/Desktop/Marketing_Analytics_project/unimib_snowit_project/data_loaded/order_details.pkl
